# Aerosol Prediction Neural Network

## Data Caracterization

### Data loading

In [1]:
import pandas as pd

df = pd.read_csv("datasets/train.csv")

print(f'Number of features: {df.shape[1]}')
print(f'Number of instances: {df.shape[0]}')
df.head()

Number of features: 10
Number of instances: 10438


,id,elevation,ozone,NO2,azimuth,zenith,incidence_azimuth,incidence_zenith,file_name_l1,value_550
0,1,10,318,0.248,150.6,31.8,286.1,8.0,AAOT_45-3139_12-5083_COPERNICUS_S2_20180807T10...,0.277
1,2,10,302,0.279,161.6,44.2,243.6,3.9,AAOT_45-3139_12-5083_COPERNICUS_S2_20180916T10...,0.201
2,4,10,373,0.303,163.5,34.4,103.9,9.8,AAOT_45-3139_12-5083_COPERNICUS_S2_20190421T10...,0.169
3,5,10,342,0.271,144.7,25.3,286.2,7.9,AAOT_45-3139_12-5083_COPERNICUS_S2_20190623T10...,0.107
4,6,10,327,0.252,140.4,29.4,105.8,7.0,AAOT_45-3139_12-5083_COPERNICUS_S2_20190720T10...,0.188


As we can see the dataset is composed of 10 features and has 10438 instances or observations.

The target feature is the 'value_550', the one we want to be capable of predicting.

Then we can split the features into two groups, the numeric features, and the image feature. The features, 'id' (identification feature, is not important for the training and prediction), 'elevation', 'ozone', 'NO2', 'azimuth', 'zenith', 'incidence_azimuth', and 'incident_zenith' are the numeric features, that will be scaled for a better Neural Network Model training. At last, but not least, the feature file_name_l1 is the name associated to the image from the zone where the other features where measured.

## Data preprocessing

### Data separation (numerical and images)

In [2]:
from sklearn.preprocessing import StandardScaler
from keras.api.utils import load_img, img_to_array
import numpy as np
import os
import tifffile as tiff

# Preprocess numerical data
numerical_features = df[['elevation', 'ozone', 'NO2', 'azimuth', 'zenith', 'incidence_azimuth', 'incidence_zenith']]

# Numerical data scaling
scaler = StandardScaler()
# Applying the scaler to the values
numerical_features = scaler.fit_transform(numerical_features)

# Function to load and preprocess image data
def load_and_preprocess_image(filepath):
    img = tiff.imread(filepath)  # Resize images to a fixed size
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    return img_array

image_data = np.array([load_and_preprocess_image(os.path.join('./train/', filename)) for filename in df['file_name_l1']])

2024-06-13 01:53:58.626733: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 01:54:00.122334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Train and Validation Split

In [3]:
from sklearn.model_selection import train_test_split

# Target variable
target = df['value_550'].values

#Split data into training and validation sets
X_train_num, X_val_num, X_train_img, X_val_img, y_train, y_val = train_test_split(numerical_features, image_data, target, test_size=0.2, random_state=42)

## Neural Network Arquitecture

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, Concatenate

# class AOTModel:
#     def __init__(self, image_shape=(128, 128, 3), num_numerical_features=7):
#         self.image_shape = image_shape
#         self.num_numerical_features = num_numerical_features
#         self.model = self.build_model()
        
#     def build_model(self):
#         # Define CNN for image data
#         image_input = Input(shape=self.image_shape)
#         x = Conv2D(32, (3, 3), activation='relu')(image_input)
#         x = MaxPooling2D((2, 2))(x)
#         x = Conv2D(64, (3, 3), activation='relu')(x)
#         x = MaxPooling2D((2, 2))(x)
#         x = Flatten()(x)
#         x = Dense(64, activation='relu')(x)

#         # Define feedforward network for numerical data
#         numerical_input = Input(shape=(self.num_numerical_features,))
#         y = Dense(64, activation='relu')(numerical_input)
#         y = Dense(64, activation='relu')(y)

#         # Concatenate outputs of both networks
#         combined = Concatenate()([x, y])
#         z = Dense(64, activation='relu')(combined)
#         output = Dense(1)(z)

#         # Define the model
#         model = Model(inputs=[image_input, numerical_input], outputs=output)

#         # Compile the model
#         model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        
#         return model
    
class AOTModel:
    def __init__(self, image_shape=(19, 19, 3), num_numerical_features=7):

        # Image processing Neural Network
        self.image_input = Input(shape=image_shape)
        image_processing_network = Conv2D(32, (3, 3), activation='relu')(self.image_input)
        image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        image_processing_network = Conv2D(64, (3, 3), activation='relu')(image_processing_network)
        image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        image_processing_network = Flatten()(image_processing_network)
        image_processing_network = Dense(64, activation='relu')(image_processing_network)

        # Numerical processing Neural Network
        self.numerical_input = Input(shape=(num_numerical_features,))
        numerical_processing_network = self.numerical_input
        numerical_processing_network = Dense(64, activation='relu')(numerical_processing_network)
        numerical_processing_network = Dense(64, activation='relu')(numerical_processing_network)

        # Concatenation of both networks
        aot_network = Concatenate()([image_processing_network, numerical_processing_network])
        aot_network = Dense(64, activation='relu')(aot_network)
        aot_network = Dense(1)(aot_network)

        self.aot_network_arquitecture = aot_network
        del image_processing_network, numerical_processing_network, aot_network

    def model(self):
        model = Model(inputs= [self.image_input, self.numerical_input], outputs=self.aot_network_arquitecture)
        
        # Compile the model
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])

        return model


## Training

In [9]:
model = AOTModel()
model = model.model()

print(X_train_img[0])
# Train the model
history = model.fit([X_train_img, X_train_num], y_train, validation_data=([X_val_img, X_val_num], y_val), epochs=50, batch_size=32)

# Evaluate the model
val_loss, val_mae = model.evaluate([X_val_img, X_val_num], y_val)
print(f'Validation MAE: {val_mae}')

[[[14.01176471  9.82745098  9.05490196 ...  0.06666667 10.65490196
    9.58431373]
  [14.01176471  9.47058824  8.65882353 ...  0.06666667 11.11764706
    9.69411765]
  [14.01176471 10.3372549   9.78431373 ...  0.06666667 10.32941176
    9.10588235]
  ...
  [16.37647059 12.78039216 11.81960784 ...  0.0627451   4.41176471
    3.43529412]
  [16.37647059 19.7254902  17.99607843 ...  0.0627451   3.36078431
    2.60392157]
  [16.06666667 20.3254902  19.67843137 ...  0.0627451   1.69019608
    1.19607843]]

 [[14.01176471 13.15294118 12.29411765 ...  0.06666667  9.85098039
    9.03137255]
  [14.01176471 12.2745098  11.6627451  ...  0.06666667  8.51764706
    8.02745098]
  [14.01176471 14.54117647 14.0745098  ...  0.06666667  7.35294118
    6.20392157]
  ...
  [16.37647059 15.1254902  14.02745098 ...  0.0627451   4.36078431
    3.75294118]
  [16.37647059 15.32941176 14.87058824 ...  0.0627451   3.81176471
    3.27058824]
  [16.06666667 20.61568627 19.84705882 ...  0.0627451   2.9254902
    2.3

ValueError: Input 0 of layer "functional_7" is incompatible with the layer: expected shape=(None, 19, 19, 3), found shape=(None, 19, 19, 13)